In [3]:
import pandas as pd
import numpy as np
import time
import urllib.request
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

In [4]:
# Create Conf
conf = SparkConf().setAppName("Read Large CSV").setMaster("local[*]")

In [5]:
# Create SparkSession using the configured SparkConf
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
# url = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/neighborhood_market_tracker.tsv000.gz'
# # urllib.request.urlretrieve(url,)

# start = time.time()
# df = (spark.read.option("compression", "gzip")
#       .option("delimiter", "\t")
#       .option("header", "true")
#       .csv(url))
# # df = pd.read_csv(url, compression = 'gzip', sep='\t', on_bad_lines='skip')
# # end time
# execution = (time.time()-start)
# print('Execution time in minutes: ' + str(round(execution / 60, 2)))
# print('Num of rows: ', len(df))
# print('Num of columns: ', len(df.columns))
# df.head()


In [6]:
from pathlib import Path
downloads_path = str(Path.home() / "Downloads")

In [9]:
file_path = str(downloads_path + "\\neighborhood_market_tracker.tsv000.gz")
file_path

'C:\\Users\\monac\\Downloads\\neighborhood_market_tracker.tsv000.gz'

In [11]:
# Create SparkSession using the configured SparkConf
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Define the file path
# file_path = "neighborhood_market_tracker.tsv000"
file_path = str(downloads_path + "\\neighborhood_market_tracker.tsv000.gz")

# Read the CSV file and create a DataFrame
df = spark.read.option("header", "true").option("delimiter", "\t").csv(file_path)

In [13]:
# Select the specific columns from the filtered DataFrame
selected_columns = [
    "city",
    "region",
    "period_end",
    "new_listings",
    "homes_sold",
    "off_market_in_two_weeks",
    "median_sale_price",
    "median_dom",
    "months_of_supply"
]

df = df.select(*selected_columns)

# List of state codes to filter
state_codes_to_filter = ["MA", "IL", "CA", "NY", "TX", "FL", "CO"]

# Filter the DataFrame to include only rows where property_type is 'All Residential' and state_code is in the list
df_filtered = df.filter((col("property_type") == 'All Residential') & (col("state_code").isin(state_codes_to_filter)) & (year(col("period_end")).isin(2023)))

# Convert the PySpark DataFrame to a pandas DataFrame
df_pandas = df_filtered.toPandas()

In [14]:
df_pandas

,city,region,period_end,new_listings,homes_sold,off_market_in_two_weeks,median_sale_price,median_dom,months_of_supply
0,Laguna Niguel,"Laguna Niguel, CA - Niguel Ridge",2023-07-31,1,1,None,1080000,47,None
1,Kathleen,"Kathleen, FL - Country View Estates",2023-05-31,6,6,None,233000,6.5,None
2,Odessa,"Odessa, FL - Starkey Ranch",2023-06-30,51,53,0.47058823529411764,650000,26.5,None
3,Garland,"Garland, TX - East Garland",2023-07-31,42,35,0.45454545454545453,250000,22,None
4,San Jose,"San Jose, CA - Riverview",2023-05-31,1,1,None,1150000,50,None
...,...,...,...,...,...,...,...,...,...
120881,New York,"New York, NY - Midland Beach",2023-02-28,18,14,0.33333333333333331,459000,45.5,None
120882,Dallas,"Dallas, TX - L. O. Daniel",2023-06-30,7,5,0,400000,28,None
120883,Pacifica,"Pacifica, CA - East Sharp Park",2023-03-31,1,2,None,1240000,60,None
120884,San Jose,"San Jose, CA - Townsend",2023-05-31,1,2,None,670000,22,None


In [ ]:
import pandas as pd
import numpy as np
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# Create Conf
conf = SparkConf().setAppName("Read Large CSV").setMaster("local[*]")

# Create SparkSession using the configured SparkConf
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Define the file path
file_path = "neighborhood_market_tracker.tsv000"

# Read the CSV file and create a DataFrame
df = spark.read.option("header", "true").option("delimiter", "\t").csv(file_path)

# Select the specific columns from the filtered DataFrame
selected_columns = [
    "city",
    "region",
    "period_end",
    "new_listings",
    "homes_sold",
    "off_market_in_two_weeks",
    "median_sale_price",
    "median_dom",
    "months_of_supply"
]

df = df.select(*selected_columns)

# List of state codes to filter
state_codes_to_filter = ["MA", "IL", "CA", "NY", "TX", "FL", "CO"]

# Filter the DataFrame to include only rows where property_type is 'All Residential' and state_code is in the list
df_filtered = df.filter((col("property_type") == 'All Residential') & (col("state_code").isin(state_codes_to_filter)) & (year(col("period_end")).isin(2023)))

# Convert the PySpark DataFrame to a pandas DataFrame
df_pandas = df_filtered.toPandas()

In [15]:
df = df_pandas

In [16]:
# Define the regular expression pattern to extract state code from the region column
pattern = r',\s(.*?)\s-\s'

# Use str.extract to create a new column "state_code" with the extracted state code
df["state_code"] = df["region"].str.extract(pattern)
# save as csv
df.to_csv('../Datas/Filtered Data For Neigborhoods.csv', index=False)